In [1]:
#!pip install langchain
#!pip install faiss-cpu

import os
import pandas as pd
from scipy import spatial

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def calculated_cosine_similarity(embeddings):
    ''' Calculate pairwise similarities and return them '''
    # Calculate pairwise cosine similarity with sklearn library
    similarity_scores = cosine_similarity(embeddings)
    return similarity_scores

Similarity Check

In [3]:
embeddings = pd.read_pickle("../data/huggingface_top250_embedding.pkl")
merged = pd.read_csv("../data/top_250.csv")
merged.drop(columns=['Unnamed: 0'], inplace=True)
merged.head()

,article_id,title,paragraphs,ressort,authors,date,text_cleaned,Top_words
0,FALTER_201512234F1D30CEA9,"""Bewundere Frau Merkel über alles""",['Kinderwagen drängen sich bei der Eingangstür...,Steiermark,gerlinde pölsler,2015-12-23 00:00:00+00:00,kinderwagen drngen eingangstr wolfgang puchers...,pucher vinzidorf roma frauen pfarrhaus fr weih...
1,FALTER_2015122382119EA87A,Wildbretter,['Das Handbuch der Kolumnenschreiberei sieht v...,Falters Zoo,NaN,2015-12-23 00:00:00+00:00,handbuch kolumnenschreiberei sieht letzte glos...,vanillekipferln wildtier darf echt abfassen ad...
2,FALTER_2015122317D031535C,"Warum es ganz normal sein kann, zwei Zuhause z...","['Xenia, Fotografin, 22', 'Ich habe bis vor zw...",Stadtleben,barbara tóth,2015-12-23 00:00:00+00:00,xenia fotografin 22 zwei jahren tag zuhause ge...,vater wechselmodell eltern mutter zuhause dopp...
3,FALTER_20151223D853291AAA,Der Lerchenfisch,['Zuerst Korrektur eines Irrtums: Nicht die AM...,Stadtleben,florian holzer,2015-12-23 00:00:00+00:00,zuerst korrektur irrtums ama betrieb restauran...,eishken tel fisch austern fischgeschft estate ...
4,FALTER_201512238A0DEB6E2D,Uhudler-Orbán und Kernölamazonen,['Wann feiert ein Kabarettist eigentlich Silve...,Lexikon,"stefanie panzenböck, sara schausberger",2015-12-23 00:00:00+00:00,wann feiert kabarettist eigentlich silvester a...,silvester uhr kabarettist 2230 maurer special ...


In [4]:
cos_sim_matrix = calculated_cosine_similarity(embeddings)
cos_sim_matrix[110,110]

1.0000000000000013

Next steps:
* get the indexes for each ressort
* do the computation


## Computing the similarity among ressorts

get the indices for the ressorts into a list

In [5]:
# Ressort list

#merged_filtered = merged.dropna(subset=["ressort"]).reset_index()
#ressorts_list = merged_filtered["ressort"].unique()#.tolist()
ressorts_list = merged["ressort"].unique()#.tolist()
ressorts_list

array(['Steiermark', 'Falters Zoo', 'Stadtleben', 'Lexikon', 'Politik',
       'Medien', 'Falter & Meinung', 'Feuilleton', 'Kolumnen', nan,
       'Aus Wissenschaft und Forschung', 'Titel',
       'Innovation in der Wissenschaft', 'Wissenschaft und Sprache',
       'Erforschung von Menschen', 'ESSEN TRINKEN', 'Woche', 'Partytimer',
       'POLITIK', 'Musik', 'STEIERMARK'], dtype=object)

Goal is to get a ressort list matrix, similar to the cos_sim_martix, where rows and column indices are ressort 

In [6]:
# first create a dictionary with ressort values as key and row indices as dict value
res_dict = {}
for i, ressort in enumerate(ressorts_list):
    res_dict[ressorts_list[i]] = merged.index[merged["ressort"] == ressort].to_list()
#res_dict.items()

In [7]:
res_dict.keys()

dict_keys(['Steiermark', 'Falters Zoo', 'Stadtleben', 'Lexikon', 'Politik', 'Medien', 'Falter & Meinung', 'Feuilleton', 'Kolumnen', nan, 'Aus Wissenschaft und Forschung', 'Titel', 'Innovation in der Wissenschaft', 'Wissenschaft und Sprache', 'Erforschung von Menschen', 'ESSEN TRINKEN', 'Woche', 'Partytimer', 'POLITIK', 'Musik', 'STEIERMARK'])

In [8]:
# return the similarity value among two ressorts
def calc_ressort_sim(ressort1 : str, ressort2 : str): # input is 2x a ressort: value, such as "Steiermark"
    i_ressort1 = res_dict[ressort1]
    i_ressort2= res_dict[ressort2]
    sim_tmp = []
    for i1 in i_ressort1:
        for i2 in i_ressort2:
            sim_tmp.append(cos_sim_matrix[i1,i2])
    avg_sim = np.mean(sim_tmp)
    return(avg_sim)  
        

In [9]:
res_sim_df = pd.DataFrame(columns=ressorts_list, index= ressorts_list)
for res1 in ressorts_list:
    for res2 in ressorts_list:
        res_sim_df.loc[res1, res2] = calc_ressort_sim(res1, res2)

res_sim_df


/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,Steiermark,Falters Zoo,Stadtleben,Lexikon,Politik,Medien,Falter & Meinung,Feuilleton,Kolumnen,NaN,...,Titel,Innovation in der Wissenschaft,Wissenschaft und Sprache,Erforschung von Menschen,ESSEN TRINKEN,Woche,Partytimer,POLITIK,Musik,STEIERMARK
Steiermark,0.855272,0.766984,0.803861,0.850194,0.811525,0.840527,0.827429,0.795284,0.846719,NaN,...,0.742851,0.733944,0.665632,0.763421,0.838309,0.718276,0.702039,0.706554,0.903264,0.900439
Falters Zoo,0.766984,0.9543,0.819099,0.695844,0.793692,0.689373,0.79683,0.868338,0.87494,NaN,...,0.925538,0.939566,0.920398,0.942734,0.939041,0.943199,0.950331,0.945437,0.864591,0.865016
Stadtleben,0.803861,0.819099,0.795171,0.77578,0.786261,0.767981,0.797361,0.805152,0.838855,NaN,...,0.793462,0.793261,0.743061,0.810485,0.864092,0.7862,0.780111,0.782564,0.867292,0.867115
Lexikon,0.850194,0.695844,0.77578,0.87124,0.789073,0.858417,0.809475,0.754661,0.812279,NaN,...,0.672271,0.655621,0.57399,0.69248,0.789354,0.653625,0.627622,0.617362,0.887254,0.882374
Politik,0.811525,0.793692,0.786261,0.789073,0.789375,0.782082,0.800451,0.793348,0.830727,NaN,...,0.774192,0.771431,0.720523,0.79162,0.837467,0.753977,0.744192,0.750512,0.868319,0.866932
Medien,0.840527,0.689373,0.767981,0.858417,0.782082,0.84942,0.801432,0.745889,0.803099,NaN,...,0.669306,0.653588,0.57478,0.685696,0.779572,0.641791,0.618889,0.615101,0.871608,0.870417
Falter & Meinung,0.827429,0.79683,0.797361,0.809475,0.800451,0.801432,0.813718,0.802353,0.842892,NaN,...,0.774683,0.770779,0.714155,0.794079,0.846304,0.756184,0.744508,0.747259,0.884721,0.883163
Feuilleton,0.795284,0.868338,0.805152,0.754661,0.793348,0.745889,0.802353,0.834063,0.855186,NaN,...,0.844547,0.848623,0.810169,0.862989,0.889152,0.845554,0.843022,0.83668,0.875202,0.874412
Kolumnen,0.846719,0.87494,0.838855,0.812279,0.830727,0.803099,0.842892,0.855186,0.89418,NaN,...,0.844415,0.846013,0.792545,0.868355,0.913315,0.834429,0.829042,0.833929,0.922584,0.919852
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Next steps
* Based on the top 10 reccommendations, check how much is from the same ressort
* If required enrich the recommendations with items from other ressorts

In [10]:
# function of top_k_similarity
def get_topk_similar_articles(article_df, similarity_scores, article_index : int, k : int):
    
    # similarity scores for given article index
    article_scores = similarity_scores[article_index]
    
    sorted_indices = np.argsort(article_scores)[::-1][1:k+1]
    # index and score
    #top_articles = [(index, article_scores[index]) for index in sorted_indices if index != article_index]
    #print(sorted_indices)
    top_article_ids = article_df["article_id"][sorted_indices].tolist()
    
    return top_article_ids
        

In [11]:
same_numb = {new_key : [] for new_key in ressorts_list}
for article_index ,article_id in enumerate(merged["article_id"]):
    top10_sim_articles = get_topk_similar_articles(merged, cos_sim_matrix, article_index, k = 10)
    ressort = merged.query("article_id == @article_id")["ressort"].to_list()[0]
    sim_ressort = [] # np.array
    for article in top10_sim_articles:
        sim_ressort.append(merged.query("article_id == @article")["ressort"].to_list()[0]) # might need some modification
    n_same = sim_ressort.count(ressort)
    same_numb[ressort].append(n_same)

#same_numb   
    

In [12]:
merged.query("ressort == 'Steiermark'").shape[0]

342

In [13]:
same_avg = [np.mean(same_numb[key]) for key in same_numb.keys()]
occurance = [merged.query("ressort == @res").shape[0] for res in ressorts_list]
novelty_df = pd.DataFrame(data = {
    "ressorts" : ressorts_list,
    "same_ressorts_avg_from_top10" : same_avg,
    "ressort_occurancy" : occurance
})
novelty_df

,ressorts,same_ressorts_avg_from_top10,ressort_occurancy
0,Steiermark,4.675439,342
1,Falters Zoo,5.187500,80
2,Stadtleben,6.045819,873
3,Lexikon,7.595938,1428
4,Politik,4.458824,850
5,Medien,4.407960,402
6,Falter & Meinung,3.938611,619
7,Feuilleton,6.751780,983
8,Kolumnen,4.075949,158
9,NaN,0.769231,0


In our case one can assume, that the novelty is mainly determined by the ressort. Thus if an article from an other ressort is recommended for that given target article, then the novelty is high, as most likely an article with different topic is recommended. Therefore to check the novelty of the recommendation a data frame was created, which contains for each ressort, that in average from the top 10 recommendations, how many is from the same ressort.
Based on the table above, one can conclude, that for article that are part of a ressorts with smaller occurance, the recommended articles tend to be from a different ressort. Therefore the novelty is high, what one should rather check is the accuracy of the recommendation.
As the same time for more frequent ressorts, such as Lexikon, from the top 10 recommendations, roughly 7-8 articles are also from the ressort Lexikon and 2-3 in average from an other ressort. Therefore one can conclude that the recommendation based on cosine similarity contains a sufficiant amount of articles from different ressort even for the most frequent ressort.